# Worksheet 08

Name:  Eric Wang

UID: U21489105

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[6.581379193291866, 5.147331352979337, 4.854345041833749, 5.46591828672025, 4.1752054016213975, 4.420956480047348, 3.69541551397898, 5.569935585791718, 3.5069017399885007, 5.569778658649377]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
c2 = np.random.normal(8,1,10).tolist()
print(c2)

[8.40486159108956, 7.944154124451618, 8.144136440474634, 7.9292351703217365, 6.869767720956219, 7.906011933031747, 7.644946227180681, 6.62530964895915, 6.78978411082179, 8.760052891980552]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[8.760052891980552, 5.569778658649377, 6.78978411082179, 3.5069017399885007, 5.569935585791718, 3.69541551397898, 4.420956480047348, 4.1752054016213975, 5.46591828672025, 4.854345041833749]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Component 1:
* Mean: 5
* Stdev: 1

Component 2:
* Mean: 8
* Stdev: 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.569778658649377, 3.5069017399885007, 5.569935585791718, 3.69541551397898, 4.420956480047348, 4.1752054016213975, 5.46591828672025, 4.854345041833749]
[8.760052891980552, 6.78978411082179]
P(C_1) = 0.8,  P(C_2) = 0.2
mean_1 = 4.657307088578916,  mean_2 = 7.774918501401171
var_1 = 0.6118903348708092,  var_2 = 10.689990688861783


/Users/eric/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The means of both components are fairly close to the known parameters, however the variance/standard deviation of component 2 is very far from the correct value.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  8.760052891980552
probability of observing that point if it came from cluster 0 =  1.1267876546280874e-10
probability of observing that point if it came from cluster 1 =  0.03716110442883279
point =  5.569778658649377
probability of observing that point if it came from cluster 0 =  0.21446140150784607
probability of observing that point if it came from cluster 1 =  0.036533622731945145
point =  6.78978411082179
probability of observing that point if it came from cluster 0 =  0.0015025792864074396
probability of observing that point if it came from cluster 1 =  0.03716110442883279
point =  3.5069017399885007
probability of observing that point if it came from cluster 0 =  0.1113480250022157
probability of observing that point if it came from cluster 1 =  0.034460274335858666
point =  5.569935585791718
probability of observing that point if it came from cluster 0 =  0.21437939021595628
probability of observing that point if it came from cluster 1 =  0.036533733358017205
point = 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), 
        sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.786264759593509,  P(C_2) = 0.2137352404064911
mean_1 = 4.699299744596237,  mean_2 = 7.420094057415917
var_1 = 0.6720406185423616,  var_2 = 2.1815902630703623


Overall, the probability of C_1 vs C_2 is similar to that found by K-means, and the computed centers are also similar to the previous iteration. However, the variance of cluster 2 dropped dramatically, much closer to the true value.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  8.760052891980552
probability of observing that point if it came from cluster 0 =  7.0029453635106e-09
probability of observing that point if it came from cluster 1 =  0.1514316684164904
point =  5.569778658649377
probability of observing that point if it came from cluster 0 =  0.25656490776629043
probability of observing that point if it came from cluster 1 =  0.12762338141316196
point =  6.78978411082179
probability of observing that point if it came from cluster 0 =  0.004702883829850058
probability of observing that point if it came from cluster 1 =  0.17539220293937305
point =  3.5069017399885007
probability of observing that point if it came from cluster 0 =  0.1230010230313369
probability of observing that point if it came from cluster 1 =  0.036599042585332475
point =  5.569935585791718
probability of observing that point if it came from cluster 0 =  0.25648731236326416
probability of observing that point if it came from cluster 1 =  0.12763116755900208
point =  3.6954

The reported probability of a point coming from C_1 rose again. Now the algorithm is "clustering" nearly all points as coming from distribution 1.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
assignments = [[], []]
for x in zip(data, prob_c0_x):
    p = x[1]
    d = x[0]
    if random.random() <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : [5.569778658649377, 3.5069017399885007, 5.569935585791718, 3.69541551397898, 4.420956480047348, 4.1752054016213975, 5.46591828672025, 4.854345041833749]
points in cluster 1 : [8.760052891980552, 6.78978411082179]
